In [1]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:898: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:898: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_electricity' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_electricity.py'>

In [5]:
attribute_region = sa.model_attributes.get_other_attribute_table(sa.model_attributes.dim_region)
attribute_technology = sa.model_attributes.get_attribute_table(sa.model_attributes.subsec_name_entc)
attribute_time_slice = sa.model_attributes.get_other_attribute_table("time_slice")


In [10]:
##################################################################################################
###                                                                                            ###
###    MAKE UP A CAPACITYFACTOR TABLE FOR NOW--LATER, AT REGIONAL INFORMATION AND REAL DATA    ###
###                                                                                            ###
##################################################################################################

model_elec = ml.ElectricEnergy(sa.model_attributes)

# get table and add an hour column
df_capfactor = attribute_time_slice.table[[
    "time_slice", model_elec.field_nemomod_tg1, model_elec.field_nemomod_tg2
]].copy()
df_capfactor[model_elec.field_nemomod_id] = range(len(df_capfactor))

def get_hr(str_in: str) -> int:
    
    if "wd" in str_in:
        out = int(str_in.split("wd")[1])
    elif "we" in str_in:
        out = int(str_in.split("we")[1])
    else:
        out = -1
    return out

df_capfactor["hour"] = df_capfactor["time_slice"].apply(get_hr)

all_tg1 = list(set(df_capfactor[model_elec.field_nemomod_tg1]))
all_tg2 = list(set(df_capfactor[model_elec.field_nemomod_tg2]))
all_hour = list(set(df_capfactor["hour"]))
    
    

# set some classes for each generation type
all_gen = [x for x in attribute_technology.key_values if x.startswith("pp_")]
gen_renewable = sa.model_attributes.filter_keys_by_attribute(
    sa.model_attributes.subsec_name_entc, 
    {"renewable_energy_technology": 1}
)
gen_non_renewable = [x for x in all_gen if x not in gen_renewable]

# some shared elements
hours_temp_low = [0]
hours_temp_mid = [6, 18]
hours_temp_high = [12]

def build_hourly_dict(factor_high: float, factor_mid: float, factor_low: float) -> dict:
    dict_out = dict(zip(hours_temp_low, [factor_low for x in hours_temp_low]))
    dict_out.update(dict(zip(hours_temp_mid, [factor_mid for x in hours_temp_mid])))
    dict_out.update(dict(zip(hours_temp_high, [factor_high for x in hours_temp_high])))
    return dict_out

def get_factor(
    row_in: np.array, 
    dict_tg1: dict, 
    dict_hour: dict,
    max_factor: float = 1.0,
    min_factor: float = 0.0
    
) -> float:
    """
        row_in: a row of form tg1, hour from df_capfactor
        dict_tg1: dictionary mapping seasons (tg1) to factor
        dict_hour: dictionary mapping hours of the day (hour) to factor
        max_factor: maximum allowable factor
        min_factor: minimum allowable factor
    """
    
    season = row_in[0]
    hour = row_in[1]
    
    factor_hour = dict_hour.get(hour, 1)
    factor_season = dict_tg1.get(season, 1)
    
    return max(min_factor, min(max_factor, factor_hour*factor_season))
    
    
    

#
#"pp_geothermal"
# 
# dirunal flux 17-24%: https://www.researchgate.net/publication/264439904_The_effect_of_ambient_temperature_fluctuation_on_the_performance_of_geothermal_power_plants
# doi: http://dx.doi.org/10.1504/IJEX.2011.037216
#
field_geotherm = "pp_geothermal"
dict_geotherm_flux_season = {"summer": 1, "fall": 0.9, "winter": 0.8, "spring": 0.9}
dict_geotherm_flux_hour = build_hourly_dict(1, 0.9, 0.8)
df_capfactor[field_geotherm] = df_capfactor[[model_elec.field_nemomod_tg1, "hour"]].apply(
    get_factor,
    dict_tg1 = dict_geotherm_flux_season,
    dict_hour = dict_geotherm_flux_hour,
    axis = 1
)

#"pp_hydropower"
# going to be HIGHLY dependent on region/precip/etc.
#
field_hydropower = "pp_hydropower"
dict_hydropower_flux_season = {"summer": 1, "fall": 0.95, "winter": 0.9, "spring": 0.95}
dict_hydropower_flux_hour = build_hourly_dict(1, 1, 1)
df_capfactor[field_hydropower] = df_capfactor[[model_elec.field_nemomod_tg1, "hour"]].apply(
    get_factor,
    dict_tg1 = dict_hydropower_flux_season,
    dict_hour = dict_hydropower_flux_hour,
    axis = 1
)

#"pp_ocean"
# again, made up--try to capture tides at high/low, need to check this
field_ocean = "pp_ocean"
dict_ocean_flux_season = {}
#dict_ocean_flux_hour = 0.9+0.1*np.array([np.cos((x/6)*np.pi) for x in range(24)])
#dict_ocean_flux_hour = dict(zip(range(len(dict_ocean_flux_hour)), list(dict_ocean_flux_hour)))
dict_ocean_flux_hour = {0: 0.8, 6: 1, 12: 0.8, 18: 1}
df_capfactor[field_ocean] = df_capfactor[[model_elec.field_nemomod_tg1, "hour"]].apply(
    get_factor,
    dict_tg1 = dict_ocean_flux_season,
    dict_hour = dict_ocean_flux_hour,
    axis = 1
)

#
# "pp_solar"
# making this up, HIGHLY dependent on region
#
field_solar = "pp_solar"
dict_solar_flux_season = {"summer": 1, "fall": 0.9, "winter": 0.8, "spring": 0.9}
dict_solar_flux_hour = build_hourly_dict(1, 0.9, 0)
df_capfactor[field_solar] = df_capfactor[[model_elec.field_nemomod_tg1, "hour"]].apply(
    get_factor,
    dict_tg1 = dict_solar_flux_season,
    dict_hour = dict_solar_flux_hour,
    axis = 1
)

#"pp_wind"
field_wind = "pp_wind"
dict_wind_flux_season = {"summer": 0.85, "fall": 0.925, "winter": 0.925, "spring": 1}
dict_wind_flux_hour = build_hourly_dict(0.8, 0.95, 1)
df_capfactor[field_wind] = df_capfactor[[model_elec.field_nemomod_tg1, "hour"]].apply(
    get_factor,
    dict_tg1 = dict_wind_flux_season,
    dict_hour = dict_wind_flux_hour,
    axis = 1
)


# format
df_capfactor.drop([model_elec.field_nemomod_tg1, model_elec.field_nemomod_tg2, "hour"], axis = 1, inplace = True)

model_elec = ml.ElectricEnergy(sa.model_attributes)
df_capfactor = model_elec.add_index_field_region(
    df_capfactor,
    restriction_regions = attribute_region.key_values,
    restrict_to_config_region = False
).rename(
    columns = {"time_slice": model_elec.field_nemomod_time_slice}
).sort_values(
    by = [
        model_elec.field_nemomod_region,
        model_elec.field_nemomod_id
    ]
).drop(
    [model_elec.field_nemomod_id],
    axis = 1
).reset_index(
    drop = True
)


df_capfactor.to_csv(sa.dict_fp_csv_nemomod[sa.model_attributes.table_nemomod_capacity_factor], index = None, encoding = "UTF-8")



/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:898: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
